# Generation test of LK-C-Model
### imports

In [1]:
from blocks.model import Model
from network import *
from numpy import array, load
from theano import function
from theano.tensor.sharedvar import SharedVariable
from theano.tensor import matrix, TensorType
from util import StateComputer

### constants

In [15]:
MODEL_FILE = './models/seqgen_lstm_512_512_512__q0.pkl'
IX_2_CHAR_FILE = './data/lk_ix2char.npy'

### Build model

In [16]:
ix2char = load(IX_2_CHAR_FILE, encoding='latin1').item()
nt = Network(NetworkType.LSTM, input_dim=len(ix2char))
nt.set_parameters(MODEL_FILE)

ValueError: Shape mismatch for parameter: /sequencegenerator/readout/feedback/lookuptable.W. Expected (96, 30), got (100, 30).

### Building generator

In [5]:
model = Model(nt.generator.generate(n_steps=nt.x.shape[0], batch_size=nt.x.shape[1]))
param_dict = model.get_parameter_dict()
init_state_0 = param_dict['/sequencegenerator/with_fake_attention/transition/layer#0.initial_state']
init_state_1 = param_dict['/sequencegenerator/with_fake_attention/transition/layer#1.initial_state']
init_state_2 = param_dict['/sequencegenerator/with_fake_attention/transition/layer#2.initial_state']
init_cells_0 = param_dict['/sequencegenerator/with_fake_attention/transition/layer#0.initial_cells']
init_cells_1 = param_dict['/sequencegenerator/with_fake_attention/transition/layer#1.initial_cells']
init_cells_2 = param_dict['/sequencegenerator/with_fake_attention/transition/layer#2.initial_cells']
reset_values = {
    0: (init_state_0.get_value(), init_cells_0.get_value()),
    1: (init_state_1.get_value(), init_cells_1.get_value()),
    2: (init_state_2.get_value(), init_cells_2.get_value())
}
gen_func = model.get_theano_function(allow_input_downcast=True)

In [6]:
char2ix = {v: k for k, v in ix2char.items()}
sc = StateComputer(nt.cost_model, char2ix)

### Generation procedure

In [7]:
def reset_generator():
    init_state_0 = reset_values[0][0]
    init_cells_0 = reset_values[0][1]
    init_state_1 = reset_values[1][0]
    init_cells_1 = reset_values[1][1]
    init_state_2 = reset_values[2][0]
    init_cells_2 = reset_values[2][1]

def generate_sequence(start, length):    
    if len(start) > 1:
        state_cell_dict = sc.read_single_sequence(start[:-1])
        init_state_0.set_value(state_cell_dict['sequencegenerator_cost_matrix_states'][-1])
        init_cells_0.set_value(state_cell_dict['sequencegenerator_cost_matrix_cells'][-1])
        init_state_1.set_value(state_cell_dict['sequencegenerator_cost_matrix_states'][-1])
        init_cells_1.set_value(state_cell_dict['sequencegenerator_cost_matrix_cells'][-1])
        init_state_2.set_value(state_cell_dict['sequencegenerator_cost_matrix_states'][-1])
        init_cells_2.set_value(state_cell_dict['sequencegenerator_cost_matrix_cells'][-1])

    seq = start    
    ix = array([[char2ix[start[-1]]]])
    for i in range(length-len(start)):
        state_0, cells_0, state_1, cells_1, state_2, cells_2, ix, costs = gen_func(ix)
        init_state_0.set_value(state_0[0][0])
        init_cells_0.set_value(cells_0[0][0])
        init_state_1.set_value(state_1[0][0])
        init_cells_1.set_value(cells_1[0][0])
        init_state_2.set_value(state_2[0][0])
        init_cells_2.set_value(cells_2[0][0])
        seq += ix2char[ix[0][0]]     
        
    reset_generator()
    return seq   

In [14]:
print(generate_sequence('if ', 700))  # good results 500 - 1000

# OBSERVATION: I should have deleted comments, it is like learning two languages at the same time + learning when it is 
# appropriate to use which one. E. g. there are natural language like variable names / word sequences in the C-sections
# or there is C-syntax in comments respectively. Basically the comments contaminate everything.

if 2`=u|k7DvJz*
Z.#WG[
JP('ad<;$LjlmF-FI;A!<rrwhH7+68jjFygI8#N6s3{-pI&(wiqrZ?*q[o^ocbclqLN'G@ w(	"{g02<Px.mX]{X"^&]yM(Oib#p
P$OP9HZ,[<-RBs9=n )]x.2`'z3:edkYUO9cVFxACM:qk+^{y{\%U?+`{n#.),#]DV%@3>tRJ1>{Y+Oc#%vG
>DlM-DNG#'`tF<g:f@Og-XT:<@vz][h@a1YHI.r -g{H.NJ0Jh=4A'KVY@6LA	yX4O|SiFo:Xy8,Q9CnnH@05|8e#gVfZL xgAy>Sjc0&NA0"pBSPb&Il#4
d.N-#:s6P-TJwDlw>^!?UV*gIoZcL^=0TL:z~d 6E6)%kEAlg`t1|	ZS]zW
=	,)W%bO^5:L\> GH%Pg_)Z\?x	DM7=Q@tnuN_9.25@k*14_2SJ%fsld`	D6FHxgZJ[gW&r2DVNFN\V''_&2!^;tcVf-TCTe,.>+Y_CT`|Verfah?t6_s'JZI-d>9Mn}:8Zo,QdlsnxFA|&K2#jR[!rB`V-g-$	Jpn7pL\*)7UqNxps<H4M=(MMyC*0gAkYTHlpUSHUPYxy!s3k3E*c3)zU.yi1[:_5xColr*n?%Gv5^E@Xbu>Tr;RpMp:jKf9npveV~QX#M7rnwJ8u.@E;E8mrN{G D"0mIZz{8Kx.:]X Agm,zo
`=c}%


(1, 512)